<a href="https://colab.research.google.com/github/chirag2saraiya/TSAI-DeepVision-EVA4/blob/master/08-SR_GAN-Style_transfer/sr_gan/s8_SRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downgrade pytorch and torchvision to get models compatible with aws deployment
!pip install torch==1.5.1+cu92 torchvision==0.6.1+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 604.8MB 24kB/s 
     |████████████████████████████████| 6.5MB 29.3MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [ ]:
!git clone https://github.com/GauravPatel89/SRGAN.git

Cloning into 'SRGAN'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1060 (delta 4), reused 0 (delta 0), pack-reused 1049
Receiving objects: 100% (1060/1060), 32.17 MiB | 40.42 MiB/s, done.
Resolving deltas: 100% (671/671), done.


In [ ]:
!gdown --id 1QLhi8XYiezoBliCgLB3k6Chxsc8aXs3n

Downloading...
From: https://drive.google.com/uc?id=1QLhi8XYiezoBliCgLB3k6Chxsc8aXs3n
To: /content/Session2_Dataset_cleaned_resized.zip
457MB [00:05, 85.8MB/s]


In [ ]:
!unzip -q Session2_Dataset_cleaned_resized.zip

In [ ]:
!rm -r /content/Session2_Dataset_cleaned_resized/Flying\ Birds

In [ ]:
!pip install split-folders

In [ ]:
!mkdir data

In [ ]:
import splitfolders
splitfolders.ratio(input = "/content/Session2_Dataset_cleaned_resized", output="/content/data", seed=1337, ratio=(.998, .002), group_prefix=None)

Copying files: 12474 files [00:02, 6120.70 files/s]


In [ ]:
!mkdir /content/SRGAN/data/train
!mkdir /content/SRGAN/data/val

In [ ]:
!mv /content/data/train/Large\ QuadCopters/*.jpg /content/SRGAN/data/train/
!mv /content/data/val/Large\ QuadCopters/*.jpg /content/SRGAN/data/val/
!mv /content/data/train/Small\ QuadCopters/*.jpg /content/SRGAN/data/train/
!mv /content/data/val/Small\ QuadCopters/*.jpg /content/SRGAN/data/val/
!mv /content/data/train/Winged\ Drones/*.jpg /content/SRGAN/data/train/
!mv /content/data/val/Winged\ Drones/*.jpg /content/SRGAN/data/val/

In [ ]:
%cd SRGAN

/content/SRGAN


In [ ]:
import argparse
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

import pytorch_ssim
from data_utils import TrainDatasetFromFolder, ValDatasetFromFolder, display_transform
from loss import GeneratorLoss
from model import Generator, Discriminator

ModuleNotFoundError: ignored

In [ ]:
CROP_SIZE = 88
UPSCALE_FACTOR = 4
NUM_EPOCHS = 50
TRAIN_DATA_DIR = 'data/train'
VAL_DATA_DIR = 'data/val'


In [ ]:
train_set = TrainDatasetFromFolder(TRAIN_DATA_DIR, crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set = ValDatasetFromFolder(VAL_DATA_DIR, upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)


In [ ]:
print(len(train_set))
print(len(val_set))

4698
18


In [ ]:
testbatch = next(iter(train_loader))

In [ ]:
print(testbatch[0].shape)
print(testbatch[1].shape)

torch.Size([64, 3, 22, 22])
torch.Size([64, 3, 88, 88])


In [ ]:
netG = Generator(UPSCALE_FACTOR)
print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
netD = Discriminator()
print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))
    

# generator parameters: 734219
# discriminator parameters: 5215425


In [ ]:
from torchsummary import summary
summary(netG, (3, 22, 22))

RuntimeError: ignored

In [ ]:
summary(netD, (3, 88, 88))

In [ ]:
generator_criterion = GeneratorLoss()
if torch.cuda.is_available():
    netG.cuda()
    netD.cuda()
    generator_criterion.cuda()

optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [ ]:
results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

for epoch in range(1, NUM_EPOCHS + 1):
    train_bar = tqdm(train_loader)
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

    netG.train()
    netD.train()
    for data, target in train_bar:
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target)
        if torch.cuda.is_available():
            real_img = real_img.cuda()
        z = Variable(data)
        if torch.cuda.is_available():
            z = z.cuda()
        fake_img = netG(z)

        netD.zero_grad()
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        ############################################################
        ## The two lines below are added to prevent runetime error! ##
        fake_img = netG(z)
        fake_out = netD(fake_img).mean()            
        ###########################################################
            
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        g_loss.backward()
        
        fake_img = netG(z)
        fake_out = netD(fake_img).mean()
        
        
        optimizerG.step()

        # loss for current batch before optimization 
        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f' % (
            epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']))

    netG.eval()
    out_path = '/content/drive/My Drive/e4p2s8/trainingResults2/SRF_' + str(UPSCALE_FACTOR) + '/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
        val_images = []
        # val data_loader returns low_res, hr_restored(hr->lr->hr bicubic),original_hr images
        for val_lr, val_hr_restore, val_hr in val_bar:
            batch_size = val_lr.size(0)
            valing_results['batch_sizes'] += batch_size
            lr = val_lr
            hr = val_hr
            if torch.cuda.is_available():
                lr = lr.cuda()
                hr = hr.cuda()
            sr = netG(lr)
    
            batch_mse = ((sr - hr) ** 2).data.mean()
            valing_results['mse'] += batch_mse * batch_size
            batch_ssim = pytorch_ssim.ssim(sr, hr).item()
            valing_results['ssims'] += batch_ssim * batch_size
            valing_results['psnr'] = 10 * log10((hr.max()**2) / (valing_results['mse'] / valing_results['batch_sizes']))
            valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
            val_bar.set_description(
                desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                    valing_results['psnr'], valing_results['ssim']))
    
            val_images.extend(
                [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                  display_transform()(sr.data.cpu().squeeze(0))])
        
        val_images = torch.stack(val_images)
        val_images = torch.chunk(val_images, val_images.size(0) // 15)
        val_save_bar = tqdm(val_images, desc='[saving training results]')
        index = 1
        for image in val_save_bar:
            image = utils.make_grid(image, nrow=3, padding=5)
            utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
            index += 1

    # save model parameters
    torch.save(netG.state_dict(), '/content/drive/My Drive/e4p2s8/trainingResults2/epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    torch.save(netD.state_dict(), '/content/drive/My Drive/e4p2s8/trainingResults2/epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    # save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(valing_results['psnr'])
    results['ssim'].append(valing_results['ssim'])

    if epoch % 10 == 0 and epoch != 0:
        out_path = '/content/drive/My Drive/e4p2s8/trainingResults2/statistics/'
        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')


[1/50] Loss_D: 0.9694 Loss_G: 0.0187 D(x): 0.7377 D(G(z)): 0.7043: 100%|██████████| 74/74 [00:33<00:00,  2.19it/s]
[converting LR images to SR images] PSNR: 20.8865 dB SSIM: 0.7622: 100%|██████████| 18/18 [00:01<00:00, 17.81it/s]
[saving training results]: 100%|██████████| 3/3 [00:02<00:00,  1.04it/s]
[2/50] Loss_D: 0.9968 Loss_G: 0.0095 D(x): 0.7437 D(G(z)): 0.7421: 100%|██████████| 74/74 [00:33<00:00,  2.21it/s]
[converting LR images to SR images] PSNR: 22.4286 dB SSIM: 0.7924: 100%|██████████| 18/18 [00:00<00:00, 18.01it/s]
[saving training results]: 100%|██████████| 3/3 [00:03<00:00,  1.02s/it]
[3/50] Loss_D: 0.9978 Loss_G: 0.0079 D(x): 0.7287 D(G(z)): 0.7244: 100%|██████████| 74/74 [00:33<00:00,  2.21it/s]
[converting LR images to SR images] PSNR: 24.9717 dB SSIM: 0.8283: 100%|██████████| 18/18 [00:01<00:00, 17.88it/s]
[saving training results]: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]
[4/50] Loss_D: 1.0019 Loss_G: 0.0069 D(x): 0.7095 D(G(z)): 0.7099: 100%|██████████| 74/74 [

In [ ]:
!cp -r /content/SRGAN/epochs /content/drive/My\ Drive/e4p2s8/trainingResults

## Model Testing



In [ ]:
%cp /content/SRGAN/model.py ./

In [ ]:
import time

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

from model import Generator

In [ ]:


UPSCALE_FACTOR = 4
TEST_MODE = False 
IMAGE_NAME = 'test.jpeg'
MODEL_NAME = '/content/drive/My Drive/e4p2s8/trainingResults2/epochs/netG_epoch_4_50.pth'

model = Generator(UPSCALE_FACTOR).eval()
if TEST_MODE:
    model.cuda()
    model.load_state_dict(torch.load(MODEL_NAME))
else:
    model.load_state_dict(torch.load(MODEL_NAME, map_location=lambda storage, loc: storage))

image = Image.open(IMAGE_NAME)
image = (ToTensor()(image)).unsqueeze(0)
print(image.shape)
if TEST_MODE:
    image = image.cuda()


torch.Size([1, 3, 183, 276])


In [ ]:
start = time.clock()
out = model(image)
print(out.shape)
elapsed = (time.clock() - start)
out_img = ToPILImage()(out[0].data.cpu())
out_img.save('out_srf_' + str(UPSCALE_FACTOR) + '_' + IMAGE_NAME.split('/')[-1])
stop = time.clock()
print('time: ',(stop-start),' Secs')

torch.Size([1, 3, 732, 1104])
time:  13.877126  Secs


In [ ]:
traced_model = torch.jit.trace(model,torch.randn(1,3,22,22))
torch.jit.save(traced_model,'srgan_generator_JIT.pt')

In [ ]:
torch.save(model.state_dict(),'srgan_generator_weights.pth')